# Light Curves Simulation for all SNe types

In [1]:
import snsim
import importlib
import matplotlib.pyplot as plt
import numpy as np

#secutiry check for numpy parallelization when using dask
def limit_numpy(nthreads=1):
    """ """
    import os
    threads = str(nthreads)
    print(f"threads {threads}")
    os.environ["NUMEXPR_NUM_THREADS"] = threads
    os.environ["OMP_NUM_THREADS"] = threads
    os.environ["OPENBLAS_NUM_THREADS"] = threads
    os.environ["MKL_NUM_THREADS"] = threads
    os.environ["VECLIB_MAXIMUM_THREADS"] = threads
limit_numpy(1)

import dask
import pandas as pd

threads 1


### we use dask to speed up the simulation

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.30 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34489,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.30 GiB
Comm: tcp://127.0.0.1:41781,Total threads: 2
Dashboard: http://127.0.0.1:43113/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:40007,


### init the survey

In [3]:
survey_conf = {
        'survey_file': './survey_file_example.csv',
        'sig_psf': 0.0,
        'sig_zp': 0.01,
        'zp': 25,
        'gain': 1,
        'ra_size': 7.295,
        'dec_size': 7.465,
        'noise_key': ['skynoise', 'skysigADU'], 
         }

### init the generator obejct for all SNe

In [4]:
sniax_gen = {
    
    'rate': 'ptf19_pw',
    
    'model_name': 'plasticc'}


        

# for simplicity we use same configuration for SN core collapse
snia91bg_gen = {
    
    'rate': 'ptf19_pw',
    
    'model_name': 'plasticc'}

cosmology = {'name':'planck18'}



cmb = {'v_cmb': 0,
       'l_cmb':0,
       'b_cmb':0}

mw_dust = {'model': 'CCM89',}

#distribution of peculiar velocities of SNe
vpec_dist = {'mean_vpec':0,
             'sig_vpec':300}


## configuration dictionary, input of simulation

In [5]:
config_dic = {'data': {'write_path': './',
                       'sim_name': 'Test_simulation_SNIApec',
                       'write_format': 'parquet'},
              'survey_config': survey_conf,
              'sim_par': {'randseed': 1234, # Optional
                          'z_range': [0.01, 0.3]},
              'sniax_gen': sniax_gen,
              'snia_91bg_gen': snia91bg_gen,
              'cosmology': cosmology,
              'mw_dust': mw_dust,
              'vpec_dist': vpec_dist,
              'cmb': cmb,
              'dask': {'use': True,
                       'nworkers': 4}#parameters to use dask, nworkers depends on your machine
             }

In [6]:
#config dask
cluster.scale(config_dic['dask']['nworkers'])

## Run the simulation

In [7]:
%%time
#initialize simulator object
#first time a bit long sncosmo has to download all the Templates
sim = snsim.Simulator(config_dic)

CPU times: user 3min 47s, sys: 3.47 s, total: 3min 50s
Wall time: 3min 45s


In [8]:
sim.generators

In [9]:
# -- Generate n base param
param_tmp = sim.generators[0].gen_basic_par(1000, 1234, 
                                            min_max_t=True)

In [10]:
epochs, params = sim.survey.get_observations(
            param_tmp,
            phase_cut=None,
            nep_cut=sim.nep_cut,
            IDmin=0,
            use_dask=sim.config['dask']['use'],
            npartitions=sim.config['dask']['nworkers'])

In [11]:
#run the simulation
sim.simulate()

      _______..__   __.         _______. __  .___  ___. 
     /       ||  \ |  |        /       ||  | |   \/   | 
    |   (----`|   \|  |       |   (----`|  | |  \  /  | 
     \   \    |  . `  |        \   \    |  | |  |\/|  | 
 .----)   |   |  |\   |    .----)   |   |  | |  |  |  | 
 |_______/    |__| \__|    |_______/    |__| |__|  |__| 
================================= Version : 0.4.5+dev ====== 
-----------------------------------------------------------

SIM NAME : Test_simulation_SNIApec
CONFIG FILE : No config file
SIM WRITE DIRECTORY : ./
SIMULATION RANDSEED : 1234

-----------------------------------------------------------

SURVEY FILE : ./survey_file_example.csv

First day in survey_file : 58000.00 MJD / 2017-09-04 00:00:00.000
Last day in survey_file : 58100.00 MJD / 2017-12-13 00:00:00.000

Survey effective duration is 100.00 days

Survey envelope area is 57.74 squared degrees (0.1 % of the sky)

No cut on survey file.

----------------------------------------------------

2 SNIax lcs generated in 0.7 seconds
Sim file write in 0.0 seconds
2 SNIa91bg lcs generated in 0.7 seconds
Sim file write in 0.0 seconds

-----------------------------------------------------------

OUTPUT FILE(S) : 
- ./Test_simulation_SNIApec_SNIax.parquet
- ./Test_simulation_SNIApec_SNIa91bg.parquet



### After the simulations

In [12]:
#the sim object has the attribute samples, which contain the data of all the object we simulated
sim.samples
#one sim.sample object for each generator 

In [13]:
sim.samples[0].header
#header with useful parameter of each simulation

{'seed': 1234,
 'seed_key': (0,),
 'obj_type': 'SNIax',
 'rate': 'lambda z:  2.43e-5 * 0.24 * (0.6766/0.70)**3  *((1 + z)**2.7/(1 + ((1 + z) / 2.9))**5.6)',
 'model_name': ['SED-Iax-0478.dat.gz',
  'SED-Iax-0847.dat.gz',
  'SED-Iax-0078.dat.gz',
  'SED-Iax-0885.dat.gz',
  'SED-Iax-0271.dat.gz',
  'SED-Iax-0036.dat.gz',
  'SED-Iax-0599.dat.gz',
  'SED-Iax-0945.dat.gz',
  'SED-Iax-0687.dat.gz',
  'SED-Iax-0169.dat.gz',
  'SED-Iax-0907.dat.gz',
  'SED-Iax-0191.dat.gz',
  'SED-Iax-0619.dat.gz',
  'SED-Iax-0429.dat.gz',
  'SED-Iax-0842.dat.gz',
  'SED-Iax-0488.dat.gz',
  'SED-Iax-0009.dat.gz',
  'SED-Iax-0335.dat.gz',
  'SED-Iax-0367.dat.gz',
  'SED-Iax-0302.dat.gz',
  'SED-Iax-0194.dat.gz',
  'SED-Iax-0151.dat.gz',
  'SED-Iax-0984.dat.gz',
  'SED-Iax-0917.dat.gz',
  'SED-Iax-0301.dat.gz',
  'SED-Iax-0470.dat.gz',
  'SED-Iax-0188.dat.gz',
  'SED-Iax-0453.dat.gz',
  'SED-Iax-0405.dat.gz',
  'SED-Iax-0726.dat.gz',
  'SED-Iax-0148.dat.gz',
  'SED-Iax-0513.dat.gz',
  'SED-Iax-0134.dat.gz',
  'S

In [14]:
#meta attribute return the parameters of each object, same as sim_lcs.attrs
sim.samples[0].meta[0]

{'mu': 40.73449241201093,
 'zobs': 0.2671813466380153,
 'zCMB': 0.2671813466380153,
 'ID': 0,
 'zcos': 0.2669684117626841,
 'como_dist': 1106.5899633267518,
 'zpcmb': 0.0,
 't0': 58039.6641076986,
 'ra': 0.7522249939743451,
 'dec': 0.736416338218206,
 'vpec': 50.38505228453618,
 'min_t': 58007.98457403265,
 'max_t': 58293.1003770262,
 '1_zobs': 1.2671813466380153,
 'model_name': 'SED-Iax-0659.dat.gz',
 'model_version': 'plasticc',
 'sed_path': '/home/rosselli/.local/lib/python3.10/site-packages/snsim/sed_data/sniax_data/SIMSED.SNIax/',
 'E_dust': 0.0060517727210821284,
 'RV': 3.240215916810955,
 'mw_ebv': 0.0759705567097508,
 'mw_r_v': 3.1,
 'M0': -16.89201544851249,
 'mb': 23.844105607512603,
 'amplitude': 3.700011170262256e-17}

In [15]:
#return the simulated lightcurves
sim.samples[0].sim_lcs

time  fluxtrue  fluxerrtrue         flux      fluxerr  \
ID epochs                                                                  
0  0       58008.163265  0.000032   716.202148  -247.229702   716.378344   
   1       58010.204082  0.000033   558.747930   903.557792   559.617784   
   2       58012.244898  0.000018   242.029050   132.259773   242.305189   
   3       58014.285714  0.000006   107.626067    83.692444   108.016929   
   4       58016.326531 -0.000090    18.283327    34.527878    19.207003   
...                 ...       ...          ...          ...          ...   
1  45      58091.836735  0.397067   882.049237 -1088.221026   882.722572   
   46      58093.877551  0.031106   998.985083  1947.319423  1000.120076   
   47      58095.918367  0.029594   210.301422  -371.486503   211.210444   
   48      58097.959184  0.028281   903.796808   -49.264037   903.824160   
   49      58100.000000  0.027147   535.020354   947.495089   535.976122   

                 mag    magerr  zp zpsys  gain    skynoise  band  fieldID  \
ID epochs                                                                   
0  0             NaN       NaN  25    ab     1  716.202148  ztfg        1   
   1       17.610110  0.672450  25    ab     1  558.747930  ztfg        1   
   2       19.696431  1.989112  25    ab     1  242.029050  ztfr        1   
   3       20.193284  1.401296  25    ab     1  107.626067  ztfr        1   
   4       21.154575  0.603968  25    ab     1   18.283325  ztfg        1   
...              ...       ...  ..   ...   ...         ...   ...      ...   
1  45            NaN       NaN  25    ab     1  882.049012  ztfr        1   
   46      16.776407  0.557621  25    ab     1  998.985068  ztfg        1   
   47            NaN       NaN  25    ab     1  210.301351  ztfg        1   
   48            NaN       NaN  25    ab     1  903.796792  ztfg        1   
   49      17.558558  0.614176  25    ab     1  535.020328  ztfg        1   

           sig_zp  sig_psf  
ID epochs                   
0  0         0.01      0.0  
   1         0.01      0.0  
   2         0.01      0.0  
   3         0.01      0.0  
   4         0.01      0.0  
...           ...      ...  
1  45        0.01      0.0  
   46        0.01      0.0  
   47        0.01      0.0  
   48        0.01      0.0  
   49        0.01      0.0  

[96 rows x 15 columns]